This notebook demonstrates how to use the polyEN pipeline

In [4]:
from polyEN import *
from sklearn.metrics import r2_score

import scanpy as sc

### Read data
Input data is an anndata object that has been properly normalized

In [5]:
ann = sc.read_h5ad("/ocean/projects/hmcmutc/qsong/projects/lung_aging/src/github/data/step1_HLCA_nonsmoker_processed.h5ad")

/jet/home/qsong/.conda/envs/lung_aging/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Keep only these columns in anndata

In [6]:
ann.obs = ann.obs.loc[:,['donor_id',
                                       'age',
                                       'ann_level_1',
                                       'ann_level_2',
                                       'ann_level_3',
                                       'ann_level_4',
                                       'ann_level_5']
                                     ]

### Analysis
#### 1. Training fully with one cell type and test on the other cell type
PolyEN can be trained with one cell type or multiple cell types. You just need to include one or multiple cell type names in the argument `cts`. For example, you can specify `cts = ['Suprabasal','Basal resting']` to train model with `Suprabasal` cells and `Basal resting` cells.

Function `polyEN_train_full` will use all available donors annotated with the given cell types. The parameters are explained below:
- `anndata`: anndata object from `AnnData` package
- `cts`: A list of strings that include cell type names of interest
- `ct_column`: A string, the column that includes cell type annotations
- `donor_column`: A string, the column that includes donor ID annotations
- `age_column`: A string, the column that includes donor age annotations. This column should be integers
- `param_space`: A dictionary, this dictionary should include the hyper parameters to be tunned for polyEN model (alpha and l1_ratio).
- `mean_degree`: The degree of polynomial features of the gene mean expressions (default 2).
- `var_degree`: The degree of polynomial features of the gene variance expressions (default 2).
- `marker_genes`: The type of genes to be used for modeling. Can be one of the following: "all", "union", "fridman", "sasp2", "cellage", "senmayo". "all" means all available genes in anndata. All others represent senescence marker lists (default "union").
- `min_cells`: An integer, for the selected cell types, donors with cells smaller than this number will be excluded (default 50).
- `n_hyper_eval`: An integer representing number of evaluations for hyperparameter tunning (default 30).
- `n_components`: An integer representing number of PCA components if `use_pca` is set to `True`.
- `use_pca`: A boolean vairable representing whether to run PCA to generate input features.
This function will return a polyEN model object. Which has the same attributes and methods with regular scikit-learn model class

In [7]:
param_space = {
             'alpha': hp.choice('alpha', [0.001, 0.01, 0.1, 1, 10, 100]), 
             'l1_ratio': hp.uniform('l1_ratio', 0.1, 1.0)
    }

In [8]:
model = polyEN_train_full(anndata = ann,
                          ct_column = "ann_level_4",
                          cts = ['Suprabasal'],
                          donor_column = "donor_id",
                          age_column = 'age',
                          param_space = param_space,
                          mean_degree = 2,
                          var_degree = 2,
                          marker_genes = 'union',
                          min_cells = 20,
                          n_hyper_eval=30,
                          n_components=10,
                          use_pca=False
                         )

(1/27/2024 22:17:33) Generate training data ...
(1/27/2024 22:17:35) Done.
(1/27/2024 22:17:35) Training polyEN model...
(1/27/2024 22:17:42) Done.


Function generate_data can generate input data for polyEN model. We can use this function to generate test data for polyEN model. Addtional parameters are explained below:
- `n_rep`: number of replicates

In [9]:
test_data_X, test_data_Y, gene_type, use_pca, rep  = generate_data(ann = ann,
                                                                      cts = ['Basal resting'],
                                                                      donor_column = "donor_id",
                                                                      ct_column = "ann_level_4",
                                                                      age_column = "age",
                                                                      marker_genes = 'union',
                                                                      min_cells = 20,
                                                                      n_rep=1,
                                                                      mean_degree=2,
                                                                      var_degree=2,
                                                                      use_pca=False
                                                                     )[0]

Predict the ages on test data set

In [10]:
pred_age = model.predict(test_data_X)

We compute $R^2$ score for the predicted ages

In [32]:
r2_score(test_data_Y, pred_age)

0.5341689926918051

#### 2. Cross-validation on one or multiple cell types
polyEN pipeline also enables training and testing by cross validation. We first perform cross validation by leave-one-out (loo) test. Addtional parameters are explained below:
- `test_method`: A string specifying the cross-validation test method, can be either "loo" or "kfold".
- `n_jobs`: Number of parallel jobs

In [30]:
test_results = polyEN_train_cv(anndata = ann,
                ct_column = "ann_level_4",
                cts = ['Suprabasal'],
                donor_column = 'donor_id',
                age_column = 'age',
                param_space = param_space,
                mean_degree = 2,
                var_degree = 2,
                marker_genes = 'all',
                min_cells = 50,
                n_hyper_eval=30,
                n_components=10,
                use_pca=False,
                test_method = 'loo',
                n_jobs = 12
               )

(1/27/2024 13:13:38) Generate training data ...
(1/27/2024 13:13:44) Done.
(1/27/2024 13:13:44) Training and testing polyEN model...
(1/27/2024 13:13:44) Training and testing for rep 1


100%|██████████| 21/21 [00:00<00:00, 83.26it/s]


(1/27/2024 14:10:16) Training and testing for rep 2


100%|██████████| 21/21 [00:00<00:00, 123.65it/s]


(1/27/2024 15:8:44) Training and testing for rep 3


100%|██████████| 21/21 [00:00<00:00, 99.31it/s]


(1/27/2024 16:8:32) Training and testing for rep 4


100%|██████████| 21/21 [00:00<00:00, 89.97it/s]


(1/27/2024 17:4:1) Training and testing for rep 5


100%|██████████| 21/21 [00:00<00:00, 109.94it/s]


In [31]:
test_results

,R2,RMSE,rep,PCA,age_true,age_pred
0,0.270595,14.155788,0,False,"30.0,31.0,29.0,30.0,33.0,24.0,27.0,61.0,57.0,5...","34.09535,33.453598,26.798338,37.53339,23.0888,..."
1,0.287545,13.990337,1,False,"30.0,31.0,29.0,30.0,33.0,24.0,27.0,61.0,57.0,5...","33.194733,33.482086,27.944025,37.61245,22.7884..."
2,0.356571,13.295357,2,False,"30.0,31.0,29.0,30.0,33.0,24.0,27.0,61.0,57.0,5...","33.083305,28.904392,27.98568,37.49653,22.75858..."
3,0.340516,13.460205,3,False,"30.0,31.0,29.0,30.0,33.0,24.0,27.0,61.0,57.0,5...","33.081223,33.46962,36.109573,37.49811,22.88818..."
4,0.217892,14.658274,4,False,"30.0,31.0,29.0,30.0,33.0,24.0,27.0,61.0,57.0,5...","33.068184,33.465015,27.258541,36.04915,22.8757..."


We can also perform cross_validation by K fold cross validation. Addtional parameters are explained below:
- `val_ratio`: A float number representing the ratio of training data used for hyperparameter tunning (every iteration of the k fold). Default value is 0.2

In [13]:
test_results = polyEN_train_cv(anndata = ann,
                ct_column = "ann_level_1",
                cts = ['Immune'],
                donor_column = 'donor_id',
                age_column = 'age',
                param_space = param_space,
                mean_degree = 2,
                var_degree = 2,
                marker_genes = 'union',
                min_cells = 20,
                n_hyper_eval=30,
                n_components=10,
                use_pca=False,
                test_method = 'kfold',
                val_ratio = 0.2,
                k = 2,
                n_jobs = 2
               )

(1/27/2024 22:18:35) Generate training data ...
(1/27/2024 22:18:37) Done.
(1/27/2024 22:18:37) Training and testing polyEN model...
(1/27/2024 22:18:37) Training and testing for rep 1


100%|██████████| 2/2 [00:58<00:00, 29.22s/it]


(1/27/2024 22:19:35) Training and testing for rep 2


100%|██████████| 2/2 [00:43<00:00, 21.79s/it]


(1/27/2024 22:20:19) Training and testing for rep 3


100%|██████████| 2/2 [00:50<00:00, 25.31s/it]


(1/27/2024 22:21:10) Training and testing for rep 4


100%|██████████| 2/2 [00:54<00:00, 27.33s/it]


(1/27/2024 22:22:4) Training and testing for rep 5


100%|██████████| 2/2 [00:50<00:00, 25.45s/it]


In [15]:
test_results

,R2,RMSE,rep,PCA,age_true,age_pred
0,-24.250349,83.347827,0,False,"42.5,67.5,31.0,29.999999999999996,33.0,24.0,27...","43.672997,55.05066,47.042316,466.4402,-40.0749..."
1,-23.039001,81.324012,1,False,"42.5,67.5,31.0,29.999999999999996,33.0,24.0,27...","44.56045,56.561367,48.957485,459.33667,-30.241..."
2,-22.638736,80.644122,2,False,"42.5,67.5,31.0,29.999999999999996,33.0,24.0,27...","44.523834,56.492958,49.02276,455.39435,-29.184..."
3,-22.724629,80.790501,3,False,"42.5,67.5,31.0,29.999999999999996,33.0,24.0,27...","44.53233,56.50912,49.00869,456.2362,-29.427742..."
4,-23.985012,82.908750,4,False,"42.5,67.5,31.0,29.999999999999996,33.0,24.0,27...","44.470642,56.40152,48.727062,467.85547,-31.972..."
